## Use Train dataset

In [1]:
import pandas as pd
import numpy as np
import shutil
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import os

from keras.preprocessing.image import ImageDataGenerator

2022-09-30 02:10:27.590557: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
def evaluate_data(Y_test, Y_pred):
    conf_matrix = confusion_matrix(Y_pred, Y_test)

    TP = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TN = conf_matrix[1][1]

    Recall = TP / (TP + FN)
    Precision = TP / (TP + FP)
    Accuracy = (TP + TN) / (TP + FP + FN + TN)
    F1_Score = 2 * (Recall * Precision) / (Recall + Precision)
    print(conf_matrix)
    print("Accuracy : {0:.2f}      F1_Score : {1:.2f}".format(Accuracy, F1_Score))
    print("Recall : {0:.2f}        Precision : {1:.2f}".format(Recall, Precision))

## 224 X 224 resize 이미지
- train data : validation data: test data = 8 : 1 : 1

In [3]:
def test_data_model_224(model_file):
    model = tf.keras.models.load_model(model_file)
    
    Reject_Test_Data_Path = 'Train_data_set/test/1_Reject'
    Accept_Test_Data_Path = 'Train_data_set/test/0_Accept'

    file_name = []
    y_predict = []
    y_pred = []
    y_test = []

    for filename in os.listdir(Reject_Test_Data_Path):
        img = cv2.imread(os.path.join(Reject_Test_Data_Path, filename), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, dsize = (224, 224), interpolation = cv2.INTER_CUBIC)
        img = img/255
        img = img.reshape(-1, 224, 224, 1)
        y_prediction = model.predict(img)
        y_predict.append(y_prediction)
        
    
        if y_prediction < 0.5:
            y_pred.append(0)
        else:
            y_pred.append(1)
    
        y_test.append(1)
        file_name.append(filename)

    for filename in os.listdir(Accept_Test_Data_Path):
        img = cv2.imread(os.path.join(Accept_Test_Data_Path, filename), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, dsize = (224, 224), interpolation = cv2.INTER_CUBIC)
        img = img/255
        img = img.reshape(-1, 224, 224, 1)
        y_prediction = model.predict(img)
        y_predict.append(y_prediction)
    
        if y_prediction < 0.5:
            y_pred.append(0)
        else:
            y_pred.append(1)
    
        y_test.append(0)
        file_name.append(filename)
        
    return file_name, y_pred, y_test, y_predict

In [4]:
CNN_file_name, CNN_y_pred, CNN_y_test, CNN_y_prediction = test_data_model_224("train_model(224X224)/CNN(train).h5")

2022-09-30 02:10:30.861191: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-30 02:10:31.513340: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43662 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:86:00.0, compute capability: 8.6
2022-09-30 02:10:33.464398: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


1/1 [==============================] - 0s 26ms/step


2022-09-30 02:10:35.315043: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1/1 [==============================] - 0s 25ms/step


In [5]:
VGG16_file_name, VGG16_y_pred, VGG16_y_test, VGG16_y_prediction = test_data_model_224("train_model(224X224)/VGG16(train).h5")

1/1 [==============================] - 0s 28ms/step


In [6]:
ResNet50_file_name, ResNet50_y_pred, ResNet50_y_test, ResNet50_y_prediction = test_data_model_224("train_model(224X224)/ResNet50(train).h5")

1/1 [==============================] - 0s 36ms/step


In [7]:
EfficientNetB0_file_name, EfficientNetB0_y_pred, EfficientNetB0_y_test, EfficientNetB0_y_prediction = test_data_model_224("train_model(224X224)/EfficientNet-B0(train).h5")

1/1 [==============================] - 0s 43ms/step


In [8]:
Name_True = []
Name_False = []
for i in range(len(CNN_file_name)):
    if CNN_file_name[i] == EfficientNetB0_file_name[i] == VGG16_file_name[i] == ResNet50_file_name[i]:
        Name_True.append(0)
    else:
        Name_False.append(1)
print("이름 일치 개수 : {0}, 이름 틀림 개수 : {1}, 파일 개수 : {2}".format(len(Name_True), len(Name_False), len(CNN_file_name)))

이름 일치 개수 : 124, 이름 틀림 개수 : 0, 파일 개수 : 124


In [9]:
label_True = []
label_False = []
for i in range(len(CNN_file_name)):
    if CNN_y_test[i] == EfficientNetB0_y_test[i] == VGG16_y_test[i] == ResNet50_y_test[i]:
        label_True.append(0)
    else:
        label_False.append(1)
print("레이블 일치 개수 : {0}, 레이블 불일치 개수 : {1}, 파일 개수 : {2}".format(len(label_True), len(label_False), len(CNN_file_name)))

레이블 일치 개수 : 124, 레이블 불일치 개수 : 0, 파일 개수 : 124


In [10]:
for i in range(len(CNN_file_name)):
    CNN_y_prediction[i] = CNN_y_prediction[i][0][0]
    VGG16_y_prediction[i] = VGG16_y_prediction[i][0][0]
    ResNet50_y_prediction[i] = ResNet50_y_prediction[i][0][0]
    EfficientNetB0_y_prediction[i] = EfficientNetB0_y_prediction[i][0][0]

In [11]:
row_data = {"file_name" : CNN_file_name,
            "y_test" : CNN_y_test,
            "CNN_y_pred" : CNN_y_pred,
            "CNN_y_prediction" : CNN_y_prediction,
            "VGG16_y_pred" : VGG16_y_pred,
            "VGG16_y_prediction" : VGG16_y_prediction,
            "ResNet50_y_pred" : ResNet50_y_pred,
            "ResNet50_y_prediction" : ResNet50_y_prediction,
            "EfficientNetB0_y_pred" : EfficientNetB0_y_pred,
            "EfficientNetB0_y_prediction" : EfficientNetB0_y_prediction}

df = pd.DataFrame(row_data)
df.to_csv("224X224_result.csv", index = False)

## 400 X 401 원본 이미지
- train data : validation data: test data = 8 : 1 : 1

In [ ]:
def test_data_model_400(model_file):
    model = tf.keras.models.load_model(model_file)
    
    Reject_Test_Data_Path = 'Train_data_set/test/1_Reject'
    Accept_Test_Data_Path = 'Train_data_set/test/0_Accept'

    file_name = []
    y_predict = []
    y_pred = []
    y_test = []

    for filename in os.listdir(Reject_Test_Data_Path):
        img = cv2.imread(os.path.join(Reject_Test_Data_Path, filename), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, dsize = (400, 401), interpolation = cv2.INTER_CUBIC)
        img = img/255
        img = img.reshape(-1, 400, 401, 1)
        y_prediction = model.predict(img)
        y_predict.append(y_prediction)
        
    
        if y_prediction < 0.5:
            y_pred.append(0)
        else:
            y_pred.append(1)
    
        y_test.append(1)
        file_name.append(filename)

    for filename in os.listdir(Accept_Test_Data_Path):
        img = cv2.imread(os.path.join(Accept_Test_Data_Path, filename), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, dsize = (400, 401), interpolation = cv2.INTER_CUBIC)
        img = img/255
        img = img.reshape(-1, 400, 401, 1)
        y_prediction = model.predict(img)
        y_predict.append(y_prediction)
    
        if y_prediction < 0.5:
            y_pred.append(0)
        else:
            y_pred.append(1)
    
        y_test.append(0)
        file_name.append(filename)
        
    return file_name, y_pred, y_test, y_predict

In [ ]:
CNN_file_name, CNN_y_pred, CNN_y_test, CNN_y_prediction = test_data_model_400("train_model(400X401)/CNN(train).h5")

In [ ]:
VGG16_file_name, VGG16_y_pred, VGG16_y_test, VGG16_y_prediction = test_data_model_400("train_model(400X401)/VGG16(train).h5")

In [ ]:
ResNet50_file_name, ResNet50_y_pred, ResNet50_y_test, ResNet50_y_prediction = test_data_model_400("train_model(400X401)/ResNet50(train).h5")

In [ ]:
EfficientNetB0_file_name, EfficientNetB0_y_pred, EfficientNetB0_y_test, EfficientNetB0_y_prediction = test_data_model_400("train_model(400X401)/EfficientNet-B0(train).h5")

In [ ]:
Name_True = []
Name_False = []
for i in range(len(CNN_file_name)):
    if CNN_file_name[i] == EfficientNetB0_file_name[i] == VGG16_file_name[i] == ResNet50_file_name[i]:
        Name_True.append(0)
    else:
        Name_False.append(1)
print("이름 일치 개수 : {0}, 이름 틀림 개수 : {1}, 파일 개수 : {2}".format(len(Name_True), len(Name_False), len(CNN_file_name)))

In [ ]:
label_True = []
label_False = []
for i in range(len(CNN_file_name)):
    if CNN_y_test[i] == EfficientNetB0_y_test[i] == VGG16_y_test[i] == ResNet50_y_test[i]:
        label_True.append(0)
    else:
        label_False.append(1)
print("레이블 일치 개수 : {0}, 레이블 불일치 개수 : {1}, 파일 개수 : {2}".format(len(label_True), len(label_False), len(CNN_file_name)))

In [ ]:
for i in range(len(CNN_file_name)):
    CNN_y_prediction[i] = CNN_y_prediction[i][0][0]
    VGG16_y_prediction[i] = VGG16_y_prediction[i][0][0]
    ResNet50_y_prediction[i] = ResNet50_y_prediction[i][0][0]
    EfficientNetB0_y_prediction[i] = EfficientNetB0_y_prediction[i][0][0]

In [ ]:
row_data = {"file_name" : CNN_file_name,
            "y_test" : CNN_y_test,
            "CNN_y_pred" : CNN_y_pred,
            "CNN_y_prediction" : CNN_y_prediction,
            "VGG16_y_pred" : VGG16_y_pred,
            "VGG16_y_prediction" : VGG16_y_prediction,
            "ResNet50_y_pred" : ResNet50_y_pred,
            "ResNet50_y_prediction" : ResNet50_y_prediction,
            "EfficientNetB0_y_pred" : EfficientNetB0_y_pred,
            "EfficientNetB0_y_prediction" : EfficientNetB0_y_prediction}

df = pd.DataFrame(row_data)
df.to_csv("400X401_result.csv", index = False)